#### IMPORTS

### INPUT & PRE-PROCESS 
*FIRST TIME OR WHEN DATA CHANGES ONLY*

In [ ]:
# Run Once

import sqlite3
import pandas as pd
from geopy.distance import geodesic
import time
from concurrent.futures import ThreadPoolExecutor

# https://download.geonames.org/export/dump/
def setup_geonames_database(geonames_file, db_file):
    """Load GeoNames data into SQLite (one-time setup)."""
    conn = sqlite3.connect(db_file)
    df = pd.read_csv(
        geonames_file,
        sep="\t",
        header=None,
        names=["geonameid", "name", "asciiname", "alternatenames", "latitude", "longitude", "feature_class", "feature_code", "country_code", "cc2", "admin1_code", "admin2_code", "admin3_code", "admin4_code", "population", "elevation", "dem", "timezone", "modification_date"],
        low_memory=False
    )
    df.to_sql("geonames", conn, if_exists="replace", index=False)
    conn.close()

# Uncomment the following line to set up the GeoNames database (run only once)
setup_geonames_database("TH.txt", "geonames.db")

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Tuple, List

import xgboost as xgb
import catboost

from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    RocCurveDisplay,
    PrecisionRecallDisplay
)
from sklearn.inspection import permutation_importance

import matplotlib.pyplot as plt

import joblib

pd.set_option("display.max_columns", 100)

import os



#### FILES INPUT

In [2]:
# Define the folder and individual files
folder_path = "./files/"
file_list = [
    "20240301.csv.out",
    "20240302.csv.out",
    "20240303.csv.out"
    # "20240306.csv.out",
    # "20240307.csv.out",
    # "20240308.csv.out",
    # "20240309.csv.out"
]

# Define headers
headers = ["VehicleID", "gpsvalid", "lat", "lon", "timestamp", "speed", "heading", "for_hire_light", "engine_acc"]

# Read and merge files vertically
dataframes = []
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    df_temp = pd.read_csv(file_path, header=None, names=headers)
    dataframes.append(df_temp)

# Concatenate dataframes vertically
merged_df = pd.concat(dataframes, axis=0, ignore_index=True)

# Display the merged dataframe
merged_df.head()
df = merged_df.copy()

#### PRE-PROCESSING

In [3]:
# Convert timestamp to datetime and sort
df.sort_values(by="timestamp", inplace=True)
df.reset_index(drop=True, inplace=True)

df["timestamp"] = pd.to_datetime(df['timestamp'], errors='coerce')
df['timestamp'].sort_values()

# Extract hour from timestamp
df["hour"] = df["timestamp"].dt.hour

# Add weekend/weekday flag
df["is_weekend"] = df["timestamp"].dt.dayofweek >= 5

# Add part of the day
def get_part_of_day(hour):
    if 6 <= hour < 12:
        return "morning"
    elif 12 <= hour < 18:
        return "afternoon"
    elif 18 <= hour < 24:
        return "evening"
    else:
        return "night"

df["part_of_day"] = df["hour"].apply(get_part_of_day)

# Filter out rows with the timestamp '1970-01-01'
df = df[df['timestamp'].dt.date != pd.to_datetime('1970-01-01').date()]

# Convert speed to mph
df["mph"] = df["speed"] * 0.62


# Speed Analysis: Calculate average speed, maximum speed, and speed variance for each vehicle
speed_analysis = (
    df.groupby("VehicleID")["speed"]
    .agg(average_speed="mean", max_speed="max", speed_variance="var")
    .reset_index()
)

# Merge the speed analysis back to the main dataframe
df = df.merge(speed_analysis, on="VehicleID", how="left")

# Stop Duration: Calculate the duration a vehicle remains stationary (speed = 0)
# Sort by VehicleID and timestamp to ensure proper calculation
df.sort_values(by=["VehicleID", "timestamp"], inplace=True)

# Create a column to identify stationary periods
df["is_stationary"] = df["speed"] == 0

# Calculate the duration of stationary periods
df["stop_duration"] = (
    df.groupby("VehicleID")["is_stationary"]
    .apply(lambda x: x.cumsum() * x)
    .reset_index(drop=True)
)

# Replace non-stationary rows with 0 in stop_duration
df.loc[df["is_stationary"] == False, "stop_duration"] = 0

# Display the updated dataframe
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6028848 entries, 0 to 6028847
Data columns (total 18 columns):
 #   Column          Dtype         
---  ------          -----         
 0   VehicleID       object        
 1   gpsvalid        int64         
 2   lat             float64       
 3   lon             float64       
 4   timestamp       datetime64[ns]
 5   speed           int64         
 6   heading         int64         
 7   for_hire_light  int64         
 8   engine_acc      int64         
 9   hour            int32         
 10  is_weekend      bool          
 11  part_of_day     object        
 12  mph             float64       
 13  average_speed   float64       
 14  max_speed       int64         
 15  speed_variance  float64       
 16  is_stationary   bool          
 17  stop_duration   int64         
dtypes: bool(2), datetime64[ns](1), float64(5), int32(1), int64(7), object(2)
memory usage: 724.4+ MB


In [4]:
df.head()

,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,hour,is_weekend,part_of_day,mph,average_speed,max_speed,speed_variance,is_stationary,stop_duration
0,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-02-29 23:58:49,0,56,1,0,23,False,evening,0.0,0.0,0,0.0,True,1399
1,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:01:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,184
2,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:04:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,0
3,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:07:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,1060
4,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:10:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,667


In [5]:
# Handle missing values
df.fillna({
    "speed": df["speed"].mean(),
    "engine_acc": df["engine_acc"].mean(),
    "lat": df["lat"].mean(),
    "lon": df["lon"].mean()
}, inplace=True)

# Remove outliers in speed using IQR
Q1 = df["speed"].quantile(0.25)
Q3 = df["speed"].quantile(0.75)
IQR = Q3 - Q1
df = df[(df["speed"] >= Q1 - 1.5 * IQR) & (df["speed"] <= Q3 + 1.5 * IQR)]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5416865 entries, 0 to 6028847
Data columns (total 18 columns):
 #   Column          Dtype         
---  ------          -----         
 0   VehicleID       object        
 1   gpsvalid        int64         
 2   lat             float64       
 3   lon             float64       
 4   timestamp       datetime64[ns]
 5   speed           int64         
 6   heading         int64         
 7   for_hire_light  int64         
 8   engine_acc      int64         
 9   hour            int32         
 10  is_weekend      bool          
 11  part_of_day     object        
 12  mph             float64       
 13  average_speed   float64       
 14  max_speed       int64         
 15  speed_variance  float64       
 16  is_stationary   bool          
 17  stop_duration   int64         
dtypes: bool(2), datetime64[ns](1), float64(5), int32(1), int64(7), object(2)
memory usage: 692.2+ MB


In [7]:
# Cache to store already requested lat/lon results
location_cache = {}

def get_location_details(lat, lon, db_file="geonames.db"):
    """Perform reverse geocoding using GeoNames database."""
    # Check if the lat/lon pair is already in the cache
    cache_key = (lat, lon)
    if cache_key in location_cache:
        return location_cache[cache_key]

    conn = sqlite3.connect(db_file)
    query = """
    SELECT name, alternatenames, feature_class, feature_code, population 
    FROM geonames 
    WHERE latitude BETWEEN ? AND ? 
      AND longitude BETWEEN ? AND ? 
    LIMIT 1
    """
    lat_range = (lat - 0.01, lat + 0.01)  # Narrow down the search range
    lon_range = (lon - 0.01, lon + 0.01)
    df = pd.read_sql_query(query, conn, params=(lat_range[0], lat_range[1], lon_range[0], lon_range[1]))
    conn.close()

    if not df.empty:
        location_name = df.iloc[0]["name"]
        alternate_names = df.iloc[0]["alternatenames"]
        feature_class = df.iloc[0]["feature_class"]
        feature_code = df.iloc[0]["feature_code"]
        population = df.iloc[0]["population"]
        location_cache[cache_key] = (location_name, alternate_names, feature_class, feature_code, population)
        return location_name, alternate_names, feature_class, feature_code, population
    else:
        return "Unknown Location", "Unknown Alternates", "Unknown Class", "Unknown Code", "Unknown Population"

def process_row_with_delay(row):
    lat, lon = row["lat"], row["lon"]
    return get_location_details(lat, lon)

def add_location_features(df):
    location_names = []
    alternate_names = []
    feature_classes = []
    feature_codes = []
    populations = []

    with ThreadPoolExecutor(max_workers=6) as executor:  # Reduce max_workers to limit resource usage
        results = list(executor.map(process_row_with_delay, [row for _, row in df.iterrows()]))

    for location_name, alternate_name, feature_class, feature_code, population in results:
        location_names.append(location_name)
        alternate_names.append(alternate_name)
        feature_classes.append(feature_class)
        feature_codes.append(feature_code)
        populations.append(population)

    df["location_name"] = location_names
    df["alternate_names"] = alternate_names
    df["feature_class"] = feature_classes
    df["feature_code"] = feature_codes
    df["population"] = populations
    return df

In [8]:
# Create interaction features 
df["speed_engine_interaction"] = df["speed"] * df["engine_acc"]
df["hour_weekend_interaction"] = df["hour"] * df["is_weekend"]

# Create lag features for speed and stop duration
df["lag_speed"] = df.groupby("VehicleID")["speed"].shift(1)
df["lag_stop_duration"] = df.groupby("VehicleID")["stop_duration"].shift(1)

# Fill NaN values created by lag features
df.fillna({
    "lag_speed": df["speed"].mean(),
    "lag_stop_duration": 0
}, inplace=True)

df.head()

,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,hour,is_weekend,part_of_day,mph,average_speed,max_speed,speed_variance,is_stationary,stop_duration,speed_engine_interaction,hour_weekend_interaction,lag_speed,lag_stop_duration
0,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-02-29 23:58:49,0,56,1,0,23,False,evening,0.0,0.0,0,0.0,True,1399,0,0,7.002803,0.0
1,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:01:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,184,0,0,0.000000,1399.0
2,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:04:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,0,0,0,0.000000,184.0
3,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:07:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,1060,0,0,0.000000,0.0
4,++qQzutWwL31NcUo8s0jiGZzzS0,1,13.64456,100.63788,2024-03-01 00:10:49,0,56,1,0,0,False,night,0.0,0.0,0,0.0,True,667,0,0,0.000000,1060.0


In [25]:
from tqdm import tqdm
import time
import os
from IPython.display import clear_output

# Reduce amount (for testing only)
# df = df.sample(n=300000).reset_index(drop=True)

# Example usage with chunk processing
chunk_size = 5000  # Define the size of each chunk
num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size != 0 else 0)  # Calculate the number of chunks

# Initialize an empty list to store processed chunks
processed_chunks = []

for i in range(num_chunks):
    if i % 10 == 0:
        print(f"Processing chunk {i + 1} of {num_chunks}...")
    start_idx = i * chunk_size
    end_idx = min((i + 1) * chunk_size, len(df))
    
    # Process the current chunk
    chunk = df.iloc[start_idx:end_idx].copy()
    chunk = add_location_features(chunk)
    
    # Append the processed chunk to the list
    processed_chunks.append(chunk)
    
    # Estimate time left using tqdm (progress bar)
    if i == 0:
        start_time = time.time()
    elif i % 10 == 0:
        elapsed = time.time() - start_time
        avg_time_per_chunk = elapsed / (i + 1)
        chunks_left = num_chunks - (i + 1)
        est_time_left = avg_time_per_chunk * chunks_left
        print(f"Estimated time left: {est_time_left/60:.2f} minutes")
    # Clear output after every 10 chunks to keep notebook responsive
    if i % 20 == 0:
        clear_output(wait=True)


# Concatenate all processed chunks back into a single DataFrame
df = pd.concat(processed_chunks, axis=0).reset_index(drop=True)

Processing chunk 1071 of 1084...
Estimated time left: 1.40 minutes
Processing chunk 1081 of 1084...
Estimated time left: 0.32 minutes


In [30]:
df["stop_duration"] = df["stop_duration"].fillna(0).astype(int)
df = df[df["feature_code"].notnull()]

In [31]:
df.sample(n=3)

,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,hour,is_weekend,part_of_day,mph,average_speed,max_speed,speed_variance,is_stationary,stop_duration,speed_engine_interaction,hour_weekend_interaction,lag_speed,lag_stop_duration,location_name,alternate_names,feature_class,feature_code,population
922322,9Ve5AnJq7WSlZzDcVt4Xsnq67r0,1,9.51456,100.05244,2024-03-01 08:47:53,0,14,0,0,8,False,morning,0.0,12.129808,79,311.390549,True,212,0,0,0.0,766.0,Ban Ko Khwan,None,P,PPL,0
244924,1L3i0FOYtlCdKA4dU9sO4uY9j9o,1,13.68806,100.37688,2024-03-02 07:33:21,0,304,0,0,7,True,morning,0.0,14.311650,108,551.305449,True,0,0,7,0.0,0.0,Khlong Thawi Watthana,"Bang Khang,Khlong Khwang,Khlong Thawi Watthana...",H,CNL,0
4140328,lQVGBED6zitZw6M96DQk5sfGo1c,1,13.93882,100.49267,2024-03-03 02:49:27,0,3,0,0,2,True,night,0.0,0.567586,56,17.385570,True,0,0,2,0.0,0.0,Khlong Bang Tanai,"Khlong Bang Tanai,Khlong Tanai,khlxng bang ta ...",H,CNL,0


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5400797 entries, 0 to 5416864
Data columns (total 27 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   VehicleID                 object        
 1   gpsvalid                  int64         
 2   lat                       float64       
 3   lon                       float64       
 4   timestamp                 datetime64[ns]
 5   speed                     int64         
 6   heading                   int64         
 7   for_hire_light            int64         
 8   engine_acc                int64         
 9   hour                      int32         
 10  is_weekend                bool          
 11  part_of_day               object        
 12  mph                       float64       
 13  average_speed             float64       
 14  max_speed                 int64         
 15  speed_variance            float64       
 16  is_stationary             bool          
 17  stop_duration

In [ ]:
df.to_csv("processed_df_training.csv", index=False)

### TRAINING

In [7]:
df = pd.read_csv("processed_df_training.csv")
df.sample(n=3)

,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,hour,is_weekend,part_of_day,mph,average_speed,max_speed,speed_variance,is_stationary,stop_duration,speed_engine_interaction,hour_weekend_interaction,lag_speed,lag_stop_duration,location_name,alternate_names,feature_class,feature_code,population
4725387,rxh6sjH8SIRGMrB35r2It7vOFSo,1,13.83939,100.66168,2024-03-03 09:35:32,0,17,0,1,9,True,morning,0.0,13.668176,90,394.988203,True,857,0,9,23.0,0.0,Ban Baen Phichit,NaN,P,PPL,0
4245618,miXYXn1qk1OITI0Tbo5nlc5Zlv0,1,14.03363,100.63735,2024-03-02 04:55:34,0,272,1,0,4,True,night,0.0,9.862800,143,508.897460,True,0,0,4,0.0,307.0,Wat Kla Cha-um,"Wat Kla Cha-um,Wat Kla Chaum,wad kla chxum,วัด...",S,MSTY,0
5337932,zHvSNdQuzkzk2XnycU93wsGvuXI,1,13.67204,100.49789,2024-03-02 11:45:04,0,201,0,0,11,True,morning,0.0,3.331143,96,174.155887,True,0,0,11,0.0,319.0,Rat Burana,"Amphoe Rat Burana,Khet Rat Burana,Rat Burana,r...",A,ADM2,110391


In [13]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

# Define features and target
X = df[["hour", "lat", "lon", "is_weekend", "engine_acc", "gpsvalid", "speed", "is_stationary", "stop_duration", "feature_class", "feature_code", "part_of_day", "location_name"]]
y = df["for_hire_light"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

# Initialize the model
model = CatBoostClassifier(
    iterations=15000,
    learning_rate=0.025,
    depth=7,
    loss_function='CrossEntropy', #Logloss
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100,
    task_type="GPU",  # Enable GPU training
    early_stopping_rounds=100 # Stop if no improvement for 100 iterations
)

# Train the model
model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=["feature_class", "feature_code", "part_of_day", "location_name"], early_stopping_rounds=100)

# Evaluate the model
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Save the model
joblib.dump(model, "catboost_model_20k-0.025-logloss.pkl")

0:	learn: 0.7086132	test: 0.7092375	best: 0.7092375 (0)	total: 319ms	remaining: 1h 19m 39s
100:	learn: 0.7281290	test: 0.7302029	best: 0.7302418 (98)	total: 28.2s	remaining: 1h 9m 17s
200:	learn: 0.7320025	test: 0.7342708	best: 0.7342708 (200)	total: 53.9s	remaining: 1h 6m 12s
300:	learn: 0.7351029	test: 0.7375870	best: 0.7376815 (298)	total: 1m 21s	remaining: 1h 6m 16s
400:	learn: 0.7383565	test: 0.7403977	best: 0.7404273 (399)	total: 1m 49s	remaining: 1h 6m 41s
500:	learn: 0.7414507	test: 0.7435565	best: 0.7435565 (500)	total: 2m 17s	remaining: 1h 6m 30s
600:	learn: 0.7441900	test: 0.7462968	best: 0.7462968 (600)	total: 2m 46s	remaining: 1h 6m 28s
700:	learn: 0.7465390	test: 0.7487298	best: 0.7487298 (700)	total: 3m 14s	remaining: 1h 6m 17s
800:	learn: 0.7489695	test: 0.7513054	best: 0.7513054 (800)	total: 3m 43s	remaining: 1h 5m 54s
900:	learn: 0.7508851	test: 0.7531773	best: 0.7531773 (900)	total: 4m 10s	remaining: 1h 5m 21s
1000:	learn: 0.7525639	test: 0.7546623	best: 0.7546678 (9

['catboost_model_20k-0.025-logloss.pkl']

In [14]:
df.sample(3)

,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,hour,is_weekend,part_of_day,mph,average_speed,max_speed,speed_variance,is_stationary,stop_duration,speed_engine_interaction,hour_weekend_interaction,lag_speed,lag_stop_duration,location_name,alternate_names,feature_class,feature_code,population
2664821,TZfiLFTkF0JhZCyx4e6TNKGd+yY,1,7.91131,98.40281,2024-03-01 22:17:21,0,174,0,0,22,False,evening,0.0,7.951881,85,257.501280,True,0,0,0,0.0,641.0,Ban Ku Ku,"Ban Ku Ku,ban ku ku,บ้านกู้กู",P,PPL,0
3520570,eCbCt5Az7RzQZxQWCoelN5vtxXY,1,13.79761,100.55036,2024-03-02 14:18:00,0,260,1,1,14,True,afternoon,0.0,18.614475,118,710.516231,True,0,0,14,12.0,0.0,Sathani Rotfai Bang Sue,"Ban Su,Bang Su,Bang Sue,Bang Sue Station,Bangs...",S,RSTN,0
3592880,f2ybvZwVfzKYHj5+C7GYxGwRmEw,1,13.74579,100.34824,2024-03-01 22:04:11,0,73,0,0,22,False,evening,0.0,0.000000,0,0.000000,True,158,0,0,0.0,0.0,Ban Wat Sala Daeng,"Ban Sala Daeng,Ban Sala Deng,Ban Wat Sala Daen...",P,PPL,0


##### OLD

In [ ]:
# # Install CatBoost if not already installed

# from catboost import CatBoostClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Initialize the CatBoostClassifier
# catboost_model = CatBoostClassifier(
#     iterations=500,          # Number of boosting iterations
#     learning_rate=0.1,       # Learning rate
#     depth=6,                 # Depth of the trees
#     loss_function='Logloss', # Loss function
#     eval_metric='Accuracy',  # Evaluation metric
#     random_seed=42,          # Random seed for reproducibility
#     verbose=100              # Print training progress every 100 iterations
# )

# # Train the model
# catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=["part_of_day", "feature_code", "feature_class"])

# # Evaluate the model
# y_pred = catboost_model.predict(X_test)
# print("Classification Report:")
# print(classification_report(y_test, y_pred))
# print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
# # Step 1: Data Preparation
# from sklearn.model_selection import KFold
# from catboost import CatBoostClassifier

# from sklearn.metrics import classification_report, accuracy_score

# df.dropna(inplace=True)

# df["hour"] = df["timestamp"].dt.hour

# # Define features and target
# X = df[["hour", "lat", "lon", "is_weekend", "engine_acc", "gpsvalid", 'speed', 'stop_duration', 'feature_class', 'feature_code', 'part_of_day']]
# y = df["for_hire_light"]

# # Step 2: Model Training with CatBoost and K-Fold Cross-Validation
# kf = KFold(n_splits=8, shuffle=True, random_state=42)

# # Initialize the model
# model = CatBoostClassifier(
#     iterations=500,
#     learning_rate=0.1,
#     depth=6,
#     loss_function='Logloss',
#     eval_metric='Accuracy',
#     random_seed=42,
#     verbose=100
# )

# # Lists to store results
# fold_accuracies = []
# fold_reports = []

# # Perform K-Fold Cross-Validation
# for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
#     print(f"Fold {fold + 1}")
    
#     # Split the data
#     X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
#     # Train the model
#     model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=["feature_class", "feature_code", "part_of_day"])
    
#     # Evaluate the model
#     y_pred = model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     report = classification_report(y_test, y_pred)
    
#     # Store results
#     fold_accuracies.append(accuracy)
#     fold_reports.append(report)
    
#     print(f"Accuracy for Fold {fold + 1}: {accuracy}")
#     # print(f"Classification Report for Fold {fold + 1}:\n{report}")

# # Print overall results
# print("K-Fold Cross-Validation Results:")
# print(f"Mean Accuracy: {sum(fold_accuracies) / len(fold_accuracies)}")

In [ ]:
# # Step 1: Data Preparation
# from sklearn.model_selection import train_test_split, KFold
# from sklearn.preprocessing import LabelEncoder

# # Select relevant columns
# df = df[["timestamp", "lat", "lon", "for_hire_light", "is_weekend", "part_of_day"]]
# df.dropna(inplace=True)

# # Encode target variable
# label_encoder = LabelEncoder()
# df["for_hire_light"] = label_encoder.fit_transform(df["for_hire_light"])

# # Encode categorical feature 'part_of_day'
# df["part_of_day"] = label_encoder.fit_transform(df["part_of_day"])

# # Extract time-based features
# df["hour"] = df["timestamp"].dt.hour

# # Define features and target
# X = df[["hour", "lat", "lon", "is_weekend", "part_of_day"]]
# y = df["for_hire_light"]

# # Step 2: Model Training with K-Fold Cross-Validation
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report, accuracy_score

# # Initialize the model
# model = RandomForestClassifier(random_state=42)

# # Initialize K-Fold
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Lists to store results
# fold_accuracies = []
# fold_reports = []

# # Perform K-Fold Cross-Validation
# for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
#     print(f"Fold {fold + 1}")
    
#     # Split the data
#     X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
#     # Train the model
#     model.fit(X_train, y_train)
    
#     # Evaluate the model
#     y_pred = model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     report = classification_report(y_test, y_pred)
    
#     # Store results
#     fold_accuracies.append(accuracy)
#     fold_reports.append(report)
    
#     print(f"Accuracy for Fold {fold + 1}: {accuracy}")
#     print(f"Classification Report for Fold {fold + 1}:\n{report}")

# # Print overall results
# print("K-Fold Cross-Validation Results:")
# print(f"Mean Accuracy: {sum(fold_accuracies) / len(fold_accuracies)}")

### BLIND TEST

#### IMPORT
Run once

In [ ]:
# Defined blind dataset for testing later

# model = joblib.load("catboost_model.pkl")

new_data_path = "./files/blind_eval/20240331.csv.out"
blind_df = pd.read_csv(new_data_path, header=None, names=[
    "VehicleID", "gpsvalid", "lat", "lon", "timestamp", "speed", "heading", "for_hire_light", "engine_acc"
])

In [ ]:
new_df = blind_df.sample(n=200000)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 342821 to 1594374
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   VehicleID       200000 non-null  object 
 1   gpsvalid        200000 non-null  int64  
 2   lat             200000 non-null  float64
 3   lon             200000 non-null  float64
 4   timestamp       200000 non-null  object 
 5   speed           200000 non-null  int64  
 6   heading         200000 non-null  int64  
 7   for_hire_light  200000 non-null  int64  
 8   engine_acc      200000 non-null  int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 15.3+ MB


In [ ]:
# Apply the same preprocessing steps to new_df

new_df.dropna(inplace=True)

new_df.dropna(inplace=True)
new_df["timestamp"] = pd.to_datetime(new_df["timestamp"], errors="coerce")
new_df["hour"] = new_df["timestamp"].dt.hour
new_df["is_weekend"] = new_df["timestamp"].dt.dayofweek >= 5

new_df["is_stationary"] = new_df["speed"] == 0

new_df["stop_duration"] = (
    new_df.groupby("VehicleID")["is_stationary"]
    .apply(lambda x: x.cumsum() * x)
    .reset_index(drop=True)
)

# Replace non-stationary rows with 0 in stop_duration
new_df.loc[new_df["is_stationary"] == False, "stop_duration"] = 0


def get_part_of_day(hour):
    if 6 <= hour < 12:
        return "morning"
    elif 12 <= hour < 18:
        return "afternoon"
    elif 18 <= hour < 24:
        return "evening"
    else:
        return "night"

new_df["part_of_day"] = new_df["hour"].apply(get_part_of_day)

# Create interaction features
new_df["speed_engine_interaction"] = new_df["speed"] * new_df["engine_acc"]
new_df["hour_weekend_interaction"] = new_df["hour"] * new_df["is_weekend"]

# Create lag features for speed and stop duration
new_df["lag_speed"] = new_df.groupby("VehicleID")["speed"].shift(1)
new_df["lag_stop_duration"] = new_df.groupby("VehicleID")["stop_duration"].shift(1)

# Fill NaN values created by lag features
new_df.fillna({
    "lag_speed": new_df["speed"].mean(),
    "lag_stop_duration": 0
}, inplace=True)

# Example usage with chunk processing
new_chunk_size = 2500  # Define the size of each chunk
new_num_chunks = len(new_df) // chunk_size + (1 if len(new_df) % chunk_size != 0 else 0)  # Calculate the number of chunks

# Initialize an empty list to store processed chunks
new_processed_chunks = []

for i in range(new_num_chunks):
    print(f"Processing chunk {i + 1} of {new_num_chunks}...")
    start_idx = i * new_chunk_size
    end_idx = min((i + 1) * new_chunk_size, len(new_df))
    
    # Process the current chunk
    new_chunk = new_df.iloc[start_idx:end_idx].copy()
    new_chunk = add_location_features(new_chunk)
    
    # Append the processed chunk to the list
    new_processed_chunks.append(new_chunk)

# Concatenate all processed chunks back into a single DataFrame
new_df = pd.concat(new_processed_chunks, axis=0).reset_index(drop=True)

# Replace NaN in 'stop_duration' with 0 and convert to integer
new_df["stop_duration"] = new_df["stop_duration"].fillna(0).astype(int)
new_df = new_df[new_df["feature_code"].notnull()]

Processing chunk 1 of 40...
Processing chunk 2 of 40...
Processing chunk 3 of 40...
Processing chunk 4 of 40...
Processing chunk 5 of 40...
Processing chunk 6 of 40...
Processing chunk 7 of 40...
Processing chunk 8 of 40...
Processing chunk 9 of 40...
Processing chunk 10 of 40...
Processing chunk 11 of 40...


In [ ]:
new_df.sample(n=3)

,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,hour,is_weekend,is_stationary,stop_duration,part_of_day,speed_engine_interaction,hour_weekend_interaction,lag_speed,lag_stop_duration,location_name,alternate_names,feature_class,feature_code,population
98175,iygmCVANsN9CDwFq75MmAZbXEys,1,13.65858,100.42449,2024-03-31 05:43:41,0,56,1,0,5,True,True,0,night,0,5,0.0,0.0,Bang Khun Thian,"Amphoe Bang Khun Thian,Bang Khun Thian,Bangkhu...",P,PPLA2,165491
69544,IMFIuwuXWIhWvkTv7rRMxRJJUPg,1,16.71384,98.60121,2024-03-31 02:57:08,0,85,0,0,2,True,True,0,night,0,2,0.0,0.0,Rongrian Ban Hua Fai,"Rongrian Ban Hua Fai,rongreiyn ban haw fay,โรง...",S,SCH,0
95010,sOiZdxY8lWOot6w9YXZIN3F29WI,1,13.64012,100.63128,2024-03-31 02:56:05,0,201,0,0,2,True,True,0,night,0,2,0.0,0.0,Rongrian Khlong Krathum,"Rongrian Khlong Krathum,rongreiyn khlxng kra t...",S,SCH,0


In [ ]:
# Export dataframe to CSV
new_df.to_csv("processed_df_blind_test.csv", index=False)

#### EVALUATE

In [57]:
blind_input = pd.read_csv("processed_df_blind_test.csv")

In [58]:
# Evaluate the model on new_df
new_df = blind_input.sample(n=50000)

X_new = new_df[["hour", "lat", "lon", "is_weekend", "engine_acc", "gpsvalid", "speed", "is_stationary", "stop_duration", "feature_class", "feature_code", "part_of_day", "location_name"]]
y_new = new_df["for_hire_light"]
y_new_pred = model.predict(X_new)

# Evaluate on new data
print("Evaluation on New Dataset:")
print(classification_report(y_new, y_new_pred))
print("Accuracy:", accuracy_score(y_new, y_new_pred))

Evaluation on New Dataset:
              precision    recall  f1-score   support

           0       0.76      0.83      0.79     32260
           1       0.63      0.52      0.57     17740

    accuracy                           0.72     50000
   macro avg       0.69      0.67      0.68     50000
weighted avg       0.71      0.72      0.71     50000

Accuracy: 0.71892


In [ ]:
new_df.head()

,VehicleID,gpsvalid,lat,lon,timestamp,speed,heading,for_hire_light,engine_acc,hour,is_weekend,is_stationary,stop_duration,part_of_day,speed_engine_interaction,hour_weekend_interaction,lag_speed,lag_stop_duration,location_name,alternate_names,feature_class,feature_code,population
0,daQuwV88Nuy1BEyp0IMeDVQk/Rs,1,13.84799,100.50644,2024-03-31 22:04:37,0,190,0,0,22,True,True,0,evening,0,22,13.66035,0.0,Wat Lanna Bun,"Wat Lanna Bun,wadlannabuy,วัดลานนาบุญ",S,MSTY,0
1,TA6NYNXQ+7r6CIugz0Sxx9HgpRg,1,13.84409,100.66376,2024-03-31 07:49:10,0,176,1,0,7,True,True,0,morning,0,7,13.66035,0.0,Khlong Song Tawan Tok,"Canal Number Two,Khlong 2 Tawan Tok,Khlong Son...",H,CNL,0
2,7U4rpD5WX2E7WynuMCuLd5Cy1DQ,1,13.69474,100.74802,2024-03-31 19:13:14,10,12,0,1,19,True,False,0,evening,10,19,13.66035,0.0,Suvarnabhumi Airport,"Aehroport Suvarnabkhumi,Aerodrom Suvarnabumi,A...",S,AIRP,0
3,389IoJtguaIlGQjxZTR/KVE1Duw,1,13.74844,100.53440,2024-03-31 14:52:22,6,78,0,1,14,True,False,0,afternoon,6,14,13.66035,0.0,Khlong Saen Saep,"Bu Saen Saep,Bung Saen Saep,Khlong Bang Khanak...",H,CNL,0
4,VfOrD4FxI4K0NXboQ29I9JDD70Q,1,13.68424,100.61278,2024-03-31 11:01:07,31,90,1,1,11,True,False,0,morning,31,11,13.66035,0.0,San Chao Samakhom Tang Trakun Haeng Prathet Thai,San Chao Samakhom Tang Trakun Haeng Prathet Th...,S,SHRN,0


##### OLD

In [ ]:
# # IMPORT AND PRE PROCESS BLIND DATA TEST
# new_df = blind_df.sample(n=30000).reset_index(drop=True)

# new_df.dropna(inplace=True)
# new_df["timestamp"] = pd.to_datetime(new_df["timestamp"], errors="coerce")
# new_df["hour"] = new_df["timestamp"].dt.hour

# new_df["is_weekend"] = new_df["timestamp"].dt.dayofweek >= 5

# def get_part_of_day(hour):
#     if 6 <= hour < 12:
#         return "morning"
#     elif 12 <= hour < 18:
#         return "afternoon"
#     elif 18 <= hour < 24:
#         return "evening"
#     else:
#         return "night"

# new_df["part_of_day"] = new_df["hour"].apply(get_part_of_day)

# new_df["part_of_day"] = label_encoder.transform(new_df["part_of_day"])

# new_df["mph"] = new_df["speed"] * 0.62

# # Speed Analysis: Calculate average speed, maximum speed, and speed variance for each vehicle
# speed_analysis = (
#     new_df.groupby("VehicleID")["speed"]
#     .agg(average_speed="mean", max_speed="max", speed_variance="var")
#     .reset_index()
# )

# # Merge the speed analysis back to the main dataframe
# new_df = new_df.merge(speed_analysis, on="VehicleID", how="left")

# # Stop Duration: Calculate the duration a vehicle remains stationary (speed = 0)
# # Sort by VehicleID and timestamp to ensure proper calculation
# new_df.sort_values(by=["VehicleID", "timestamp"], inplace=True)

# # Create a column to identify stationary periods
# new_df["is_stationary"] = new_df["speed"] == 0

# # Calculate the duration of stationary periods
# new_df["stop_duration"] = (
#     new_df.groupby("VehicleID")["is_stationary"]
#     .apply(lambda x: x.cumsum() * x)
#     .reset_index(drop=True)
# )

# # Replace non-stationary rows with 0 in stop_duration
# new_df.loc[new_df["is_stationary"] == False, "stop_duration"] = 0

# # Display the updated dataframe
# new_df.reset_index(drop=True, inplace=True)

# # Example usage with chunk processing
# new_chunk_size = 5000  # Define the size of each chunk
# new_num_chunks = len(new_df) // new_chunk_size + (1 if len(new_df) % new_chunk_size != 0 else 0)  # Calculate the number of chunks

# # Initialize an empty list to store processed chunks
# new_processed_chunks = []

# for i in range(new_num_chunks):
#     print(f"Processing chunk {i + 1} of {new_num_chunks}...")
#     start_idx = i * new_chunk_size
#     end_idx = min((i + 1) * new_chunk_size, len(new_df))
    
#     # Process the current chunk
#     new_chunk = new_df.iloc[start_idx:end_idx].copy()
#     new_chunk = add_location_features(new_chunk)

#     # Append the processed chunk to the list
#     new_processed_chunks.append(new_chunk)

# # Concatenate all processed chunks back into a single DataFrame
# new_df = pd.concat(new_processed_chunks, axis=0).reset_index(drop=True)

# new_df["feature_code"] = label_encoder.transform(new_df["feature_code"])
# new_df["feature_class"] = label_encoder.transform(new_df["feature_class"])

# new_df.sample(n=5)

In [ ]:
# # Custom decoder to handle unseen labels gracefully
# def safe_inverse_transform(encoder, values):
#     mapping = {idx: label for idx, label in enumerate(encoder.classes_)}
#     return [mapping.get(value, "Unknown") for value in values]

# # Decode 'part_of_day' back to its original labels
# new_df["part_of_day"] = safe_inverse_transform(label_encoder, new_df["part_of_day"])
# new_df["feature_code"] = safe_inverse_transform(label_encoder, new_df["feature_code"])
# new_df["feature_class"] = safe_inverse_transform(label_encoder, new_df["feature_class"])

# new_df.head()

In [ ]:
# # Predict using the trained model
# X_new = new_df[["hour", "lat", "lon", "is_weekend", "engine_acc", "gpsvalid", 'speed', 'stop_duration', 'feature_class', 'feature_code', 'part_of_day']]
# y_new = new_df["for_hire_light"]
# y_new_pred = model.predict(X_new)

# # Evaluate on new data
# print("Evaluation on New (blind) Dataset:")
# print(classification_report(y_new, y_new_pred))
# print("Accuracy:", accuracy_score(y_new, y_new_pred))

## VISUALIZATION 

In [ ]:
#  # Filter rows where 'predicted_for_hire' is 1
# predicted_hire_df = new_df[new_df["predicted_for_hire"] == 1]

# # Display the filtered dataframe
# predicted_hire_df.head()

import folium
from folium.plugins import HeatMap

# Combine predictions with coordinates
new_df["predicted_for_hire"] = y_new_pred
heatmap_data = new_df[(new_df["predicted_for_hire"] == 1)][["lat", "lon"]].values.tolist()
# Create a map centered around the mean latitude and longitude
m = folium.Map(location=[new_df["lat"].mean(), new_df["lon"].mean()], zoom_start=12)

# Add heatmap layer
HeatMap(heatmap_data).add_to(m)

m

In [ ]:
import requests

# Function to perform reverse geocoding using OpenStreetMap Nominatim API
def reverse_geocode(lat, lon):
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=14&addressdetails=1"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("display_name", "Unknown Location")
    else:
        return "Unknown Location"

# Extract hotspot coordinates
hotspot_coordinates = new_df[(new_df["predicted_for_hire"] == 1) & (new_df["is_weekend"] == True) & (new_df["part_of_day"] == "night")][["lat", "lon"]]

# Get place names for each hotspot
hotspot_locations = []
for index, row in hotspot_coordinates.iterrows():
    place_name = reverse_geocode(row["lat"], row["lon"])
    hotspot_locations.append({"latitude": row["lat"], "longitude": row["lon"], "place_name": place_name})

# Display the hotspot locations
hotspot_locations_df = pd.DataFrame(hotspot_locations)
hotspot_locations_df.head()